<a href="https://colab.research.google.com/github/khatamirock/colabs_AI_LAB/blob/main/DL_cholle_conv_cats%2Bdogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## download from KAGGLE....
### **upload kaggle.json file first ** 

In [1]:
!pip install -q kaggle
# from google.colab import files
# files.upload()

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d chetankv/dogs-cats-images

 99% 430M/435M [00:12<00:00, 41.3MB/s]
100% 435M/435M [00:12<00:00, 37.7MB/s]


In [3]:
!unzip -qq /content/dogs-cats-images.zip

# CODE!!

In [4]:
import pathlib
import os, shutil, pathlib


original_dir = pathlib.Path("dataset/training_set") 
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index): 
  for category in ("cat", "dog"):
    if category=='cat':
      cat='cats'
    else:cat='dogs'
    dir = new_base_dir / subset_name / category
    print(dir)
    os.makedirs(dir,exist_ok=True)
    fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
    for fname in fnames:
      shutil.copyfile(src=original_dir/cat/ fname,dst=dir / fname)
    

In [5]:
make_subset("train", start_index=1, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500) 
make_subset("test", start_index=1500, end_index=2500)

cats_vs_dogs_small/train/cat
cats_vs_dogs_small/train/dog
cats_vs_dogs_small/validation/cat
cats_vs_dogs_small/validation/dog
cats_vs_dogs_small/test/cat
cats_vs_dogs_small/test/dog


In [6]:
from tensorflow import keras 
from tensorflow.keras import layers


inputs = keras.Input(shape=(180, 180, 3)) 
x = layers.Rescaling(1./255)(inputs) 
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()


model.compile(loss="binary_crossentropy",
 optimizer="rmsprop",
 metrics=["accuracy"])




Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                         

In [7]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
 new_base_dir / "train",
 image_size=(180, 180),
 batch_size=32)
validation_dataset = image_dataset_from_directory(
 new_base_dir / "validation",
 image_size=(180, 180),
 batch_size=32)
test_dataset = image_dataset_from_directory(
 new_base_dir / "test",
 image_size=(180, 180),
 batch_size=32)

Found 1998 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [8]:
import numpy as np 
import tensorflow as tf
random_numbers = np.random.normal(size=(10, 6))
dataset = tf.data.Dataset.from_tensor_slices(random_numbers)
# batch_dat=dataset.batch(3) #this makes a group of the input size!!!

In [ ]:
len(dataset),len(dataset.batch(5))


In [ ]:
reshaped_dataset = dataset.map(lambda x: print(x)  )
# len(dataset),len(reshaped_dataset)



In [ ]:
for data_batch, labels_batch in train_dataset:
  # print("data batch shape:", data_batch.shape)
  print("labels batch shape:",labels_batch, labels_batch.shape)
  break

data batch shape: (32, 180, 180, 3)
labels batch shape: tf.Tensor([1 1 1 0 1 0 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 1 1 1 1 1 0], shape=(32,), dtype=int32) (32,)


In [9]:
callbacks = [
 keras.callbacks.ModelCheckpoint(
 filepath="convnet_from_scratch.keras",
 save_best_only=True,
 monitor="val_loss")
]

In [10]:
history = model.fit(
 train_dataset,
 epochs=30,
 validation_data=validation_dataset,
 callbacks=callbacks)

Epoch 1/30
63/63 [==============================] - 18s 93ms/step - loss: 0.7021 - accuracy: 0.5255 - val_loss: 0.6908 - val_accuracy: 0.5070
Epoch 2/30
63/63 [==============================] - 4s 62ms/step - loss: 0.6959 - accuracy: 0.5120 - val_loss: 0.6883 - val_accuracy: 0.6090
Epoch 3/30
63/63 [==============================] - 5s 77ms/step - loss: 0.6785 - accuracy: 0.5766 - val_loss: 0.6636 - val_accuracy: 0.5640
Epoch 4/30
63/63 [==============================] - 5s 68ms/step - loss: 0.6491 - accuracy: 0.6266 - val_loss: 0.6496 - val_accuracy: 0.6090
Epoch 5/30
63/63 [==============================] - 5s 78ms/step - loss: 0.6183 - accuracy: 0.6552 - val_loss: 0.6715 - val_accuracy: 0.5850
Epoch 6/30
63/63 [==============================] - 4s 59ms/step - loss: 0.5869 - accuracy: 0.6852 - val_loss: 0.8396 - val_accuracy: 0.6240
Epoch 7/30
63/63 [==============================] - 5s 82ms/step - loss: 0.5540 - accuracy: 0.7182 - val_loss: 0.5776 - val_accuracy: 0.6990
Epoch 8/30
6

In [ ]:
import matplotlib.pyploy as plt
accuracy=history.history['accuracy']
val_accuracy=history.history['val_accuracy']

loss=history.history['loss'
val_loss=history.history['val_loss']



